In [1]:
import data_handler_uniNER

In [2]:
uniNER_dataset_MSEQA_format = data_handler_uniNER.build_uniNER_dataset_MSEQA_format()

Using custom data configuration Universal-NER--Pile-NER-type-e81e7ad3a512a503
Found cached dataset json (/Users/andrew/.cache/huggingface/datasets/Universal-NER___json/Universal-NER--Pile-NER-type-e81e7ad3a512a503/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print(uniNER_dataset_MSEQA_format['train'][23])